# Capstone Project

# Libraries

In [2]:
# Import Library 

import re
import requests
import unicodedata
from bs4 import BeautifulSoup
from pprint import pprint
import numpy as np
from numpy import array
import pandas as pd 
import csv
import pickle
from IPython.display import display
import pickle
from collections import Counter
pd.set_option('display.max_columns', 100)
from IPython.core import display as ICD


# Define Text Normalization Function

In [3]:
def restore_windows_1252_characters(restore_string):
    """
    Replace C1 control characters in the Unicode string s by the
    characters at the corresponding code points in Windows-1252,
    where possible.
    """

def to_windows_1252(match):
    try:
        return bytes([ord(match.group(0))]).decode('windows-1252')
    except UnicodeDecodeError:
        # No character at the corresponding code point: remove it.
        return ''

    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

# Grab the Document Content

In [4]:
def grab_doc_content( brand, CIK ):
    
    company = {}
    
    company['auto'] = {}
    auto = {brand : CIK
           }
    company['auto'] = auto

    value = list(auto.copy().values())
    
    return value

In [5]:
GDC = grab_doc_content('Ford', '0000037996')

In [6]:
master_filings_dict = {}
    
file_code = {}
    
file_text = {}

Accession_Number_URL = {}

In [7]:
# URL Directory For CIK    
dir_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
dir_url_list = [dir_url.format(x) for x in GDC]

#print('Directory URL: {}'.format(dir_url_list))
doc_url_list = [] 

# FOR-loop yielding Accession Numbers from CIK/URL Directory.
for CIK_num in GDC:

    doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
    doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
    doc_url_list.append(doc_url_new)


In [8]:
def url_maker(dir_url):
    dir_url_list = [dir_url.format(x) for x in GDC]

    #print('Directory URL: {}'.format(dir_url_list))
    doc_url_list = [] 

    # FOR-loop yielding Accession Numbers from CIK/URL Directory.
    for CIK_num in GDC:

        doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
        doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
        
        doc_url_list.append(doc_url_new)
        
        url_lists = zip( dir_url_list , doc_url_list )
        
    return url_lists

In [9]:
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
for dir_url, doc_url in url_maker(url): 
    print(1, dir_url)
    print(2, doc_url)

1 https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0000037996&type=10-&dateb=&owner=include&count=100
2 https://www.sec.gov/Archives/edgar/data/0000037996/{xx}/{yy}.txt


In [10]:
Accession_Number_URL = {}
url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'

for dir_url, doc_url in url_maker(url):
    
    response = requests.get(dir_url)
    soup = BeautifulSoup(response.content, 'lxml')
    text = soup.get_text(strip=True)

    cleaned_text = re.findall('Acc-no: \d+-\d+-\d+' , text)

    accession_number = [n.replace('Acc-no: ', '') for n in cleaned_text]
    accessionnumber = [num.replace('-', '') for num in accession_number]

    accession_numbers = zip(accessionnumber, accession_number)
    
    cikk = [ cikk_.replace('CIK=', '') for cikk_ in re.findall('CIK=\d+', dir_url) ][0]
    CIKK = {cikk : accessionnumber}
    
    for (a,b) in accession_numbers: 
        master_filings_dict[b] = {}
        master_filings_dict[b]['sec_header_content'] = {}
        master_filings_dict[b]['filing_documents'] = None

        doc_url_single = doc_url.format(xx = a, yy = b)

        file_url_list = []

        

        file_url_list.append( doc_url_single )
        
        Accession_Number_URL.update({ b : file_url_list})

In [11]:
A_N = []
for AN, url in Accession_Number_URL.items():
    
    if AN == '0000037996-12-000049':
        break
    else:
        A_N.append(AN)
        print(url)
        
print(A_N, '\n')
print(len(A_N))

['https://www.sec.gov/Archives/edgar/data/0000037996/000003799620000041/0000037996-20-000041.txt']
['https://www.sec.gov/Archives/edgar/data/0000037996/000003799620000010/0000037996-20-000010.txt']
['https://www.sec.gov/Archives/edgar/data/0000037996/000003799619000078/0000037996-19-000078.txt']
['https://www.sec.gov/Archives/edgar/data/0000037996/000003799619000067/0000037996-19-000067.txt']
['https://www.sec.gov/Archives/edgar/data/0000037996/000003799619000026/0000037996-19-000026.txt']
['https://www.sec.gov/Archives/edgar/data/0000037996/000003799619000012/0000037996-19-000012.txt']
['https://www.sec.gov/Archives/edgar/data/0000037996/000003799618000084/0000037996-18-000084.txt']
['https://www.sec.gov/Archives/edgar/data/0000037996/000003799618000063/0000037996-18-000063.txt']
['https://www.sec.gov/Archives/edgar/data/0000037996/000003799618000036/0000037996-18-000036.txt']
['https://www.sec.gov/Archives/edgar/data/0000037996/000003799618000025/0000037996-18-000025.txt']
['https://

## Save each Filing.

In [12]:
QorK = {'10-Q' : [],
        '10-K' : []
       }

for acc_num, url in Accession_Number_URL.items():
    
    master_document_dict = {}
    
    # create a stop point
    if acc_num == '0000037996-12-000049':
        break

    else:
        
        # grab the response
        response = requests.get(url[0])

    # Soupify
        # pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
        soup = BeautifulSoup(response.content, 'lxml')
        filing_document = soup.find('document')
        
    # Parsing
        # Document type ->> document_id
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        # Document sequence ->> document_sequence
        document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
        # Document filename ->> document_filename
        document_filename = filing_document.filename.find(text=True, recursive=False).strip()
        # Document description ->> document_description
        document_description = filing_document.description.find(text=True, recursive=False).strip()
        
    # Storage    
        # initalize our document dictionary
        master_document_dict[document_id] = {}

        # add the different parts, we parsed up above.
        master_document_dict[document_id]['document_sequence'] = document_sequence
        master_document_dict[document_id]['document_filename'] = document_filename
        master_document_dict[document_id]['document_description'] = document_description
        
# Scraping
        # grab the text portion of the document, this will be used to split the document into pages.
        filing_doc_text = filing_document.find('text').extract()

        # find all the thematic breaks, these help define page numbers and page breaks.
        all_thematic_breaks = filing_doc_text.find_all('hr')
        
        # Locate and store page number via list comprehension.
        all_page_numbers = [thematic_break.previous_sibling.get_text(strip=True) for thematic_break in all_thematic_breaks if len(str(thematic_break))>6]
            
        # convert all thematic breaks to a string so it can be used for parsing
        all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]

        # prep the document text for splitting, this means converting it to a string.
        filing_doc_string = str(filing_doc_text)
    
        # handle the case where there are thematic breaks.
        if len(all_thematic_breaks) > 0: 

            # define the regex delimiter pattern, this would just be all of our thematic breaks.
            regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

            # split the document along each thematic break.
            split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

            # store the document itself
            master_document_dict[document_id]['pages_code'] = split_filing_string

        # handle the case where there are no thematic breaks.
        elif len(all_thematic_breaks) == 0:

            # handles so it will display correctly.
            split_filing_string = all_thematic_breaks

            # store the document as is, since there are no thematic breaks. In other words, no splitting.
            master_document_dict[document_id]['pages_code'] = [filing_doc_string]
            
        # display some information to the user.
        print('-'*80)
        print('The document {} was parsed.'.format(document_id))
        print('There was {} page(s) found.'.format(len(all_page_numbers)))
        print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))

        # store the documents in the master_filing_dictionary.
        master_filings_dict[acc_num]['filing_documents'] = master_document_dict
        
        # if document is 10-Q
        if document_id == '10-Q':
            QorK['10-Q'].append(acc_num) # add acc_num to QorK in 10-Q as key
        
        # if document is 10-K
        if document_id == '10-K':
            QorK['10-K'].append(acc_num) # add acc_num to QorK in 10-K as key
        
        del master_document_dict
        
        print('-'*80)
        print('All the documents for filing {} were parsed and stored.'.format(acc_num))
    


--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 76 page(s) found.
There was 76 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0000037996-20-000041 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 159 page(s) found.
There was 159 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0000037996-20-000010 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 73 page(s) found.
There was 73 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0000037996-19-000078 were parsed and stored.

--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 174 page(s) found.
There was 174 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0000037996-15-000013 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 76 page(s) found.
There was 76 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0000037996-14-000057 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 77 page(s) found.
There was 77 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0000037996-14-000043 were parsed and stored.

In [12]:
string_test = master_filings_dict['0000037996-20-000041']['filing_documents']['10-Q']['pages_code']
for page in string_test:
    if 'Cash and cash equivalents' in page:
        if 'CONSOLIDATED BALANCE SHEET' in page:
            print(page)
            print('~'*40)
#string_test[56]
# df01 = table(string_test, '10-Q')
# df = df01[0]
# df

<div><a id="sE1C0F790A66C554EB952CAF5379A108C"></a></div><div><div style="line-height:120%;text-align:left;padding-left:18px;text-indent:0px;font-size:10pt;"><span style="font-family:Arial;font-size:10pt;font-style:italic;">Item 1. Financial Statements (continued)</span></div></div><div><br/></div><div style="line-height:120%;text-align:center;font-size:10pt;"><span style="font-family:Arial;font-size:10pt;font-weight:bold;">FORD MOTOR COMPANY AND SUBSIDIARIES</span></div><div style="line-height:120%;text-align:center;font-size:10pt;"><span style="font-family:Arial;font-size:10pt;font-weight:bold;">CONSOLIDATED BALANCE SHEET</span></div><div style="line-height:120%;text-align:center;font-size:10pt;"><span style="font-family:Arial;font-size:10pt;font-weight:bold;">(in millions)</span></div><div style="line-height:120%;font-size:10pt;"><div style="padding-left:0px;text-indent:0px;line-height:normal;padding-top:10px;"><table cellpadding="0" cellspacing="0" style="font-family:Times New Roma

## Anchor Check Function



In [13]:
# Variations of HTML anchors.
CBS_A = 'CONSOLIDATED BALANCE SHEET'
# CBS_B = 'name="BALANCE_SHEETS"'
# CBS_D = 'BALANCE SHEETS '
CBS_C = 'Cash and cash equivalents'
# SoO_A = '<a name="Statements_of_Operations"' 
# SoO_B = 'id="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# SoO3 = 'name="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# CSoO10K = 'name="Consolidated_Statements_of_Operations"'
# CSoO10Knew = 'id="Consolidated_Statements_of_Operations"'

# CF_A = '<a name="Statements_of_Cash'
# CF_B = 'id="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CF3 = 'name="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CCF10K = 'name="Consolidated_Statements_of_Cash_Flows"'
# CCF10Knew = 'id="Consolidated_Statements_of_Cash_Flows"'

def anchor_check(page_code):
    
    # if the page has one of these anchors
    if (CBS_A in page_code) and (CBS_C in page_code):
        return True

    #elif (SoO_A in page_code) or (SoO_B in page_code) or (SoO3 in page_code) or (CSoO10K in page_code) or (CSoO10Knew in page_code) :
        #return True

    #elif (CF_A in page_code) or (CF_B in page_code) or (CF3 in page_code) or (CCF10K in page_code) or (CCF10Knew in page_code):
        #return True


    else:
        return False


## Cleaning Columns Function


In [14]:
month_dict = {'December' : 'Q4',
              'November' : 'Q4',
             'October' : 'Q4',
             'September' : 'Q3',
             'August': 'Q3',
             'July' : 'Q3',
             'June' : 'Q2',
             'May' : 'Q2',
             'April' : 'Q2',
             'March' : 'Q1',
             'Feburary' : 'Q1',
             'January' : 'Q1'}

In [15]:
col_name_variants = {
    'Inventor' : 'Inventory',
    'Total liabilities and' : 'Total liabilities and equity',
    'Common stock' : 'Common Stock',
    'Preferred stock': 'Preferred Stock',
    'Accounts.*receivable' : 'Accounts Receivable',
    'Accrued liabilities' : 'Accrued Liabilities',
    'Deferred revenue' : 'Deferred Revenue',
    'Redeemable noncontrolling interests in subsidiaries' : 'Noncontrolling interests in subsidiaries',
    'Short term investments' : 'Short term investments', 
    'Property|equipment.' : 'Property and Equipment',
    'Retained earnings' : 'Retained Earnings',
    'stockholders' : 'Shareholder Equity',
    '^Cash' : 'Cash',
    'Convertible senior notes' : 'Convertible Senior Notes'
}

In [16]:
def clean_col_name( name ):
    
    for detect, replacement in col_name_variants.items():

        if re.findall('.*' + detect +'.*', name, flags=re.I):

            n_column = name.replace(name, replacement) 

            return n_column 
            
    return name

In [17]:
# Throw column list-like object into cleaning function. 
def cleaning_column(r_column, m):
     
    column_list = []
    column_name = []
    
    for element_post, element in enumerate(r_column):
        try:
            ele= element.encode('cp1252').decode("utf-8", "ignore")
                
        except:
            ele= element.encode('latin1').decode("utf-8", "ignore")
            
        if ele =='':
            continue
            
        ele_norm = unicodedata.normalize('NFKD', ele)
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:
#             print(element)
            
            ele_norm = ele_norm.replace('net', '')
            
            ele_norm = ele_norm.replace( '-', ' ')
            
            s_ele_norm = re.sub(r"^\s+|\s+$", "", ele_norm)
            
            name = re.sub(r'(?=,|:|\.|\(){1}.*', repl='', string= s_ele_norm, flags=re.I)
            
            # clean_col_name function Choose old_if or new_dict
            clean_name = clean_col_name( name )
            ###
            column_name.append(clean_name)
            ###
            column_list.append(clean_name)
        
        # Handling the all the remaining "rows" in the table i.e. data/numbers.
        elif ele_norm not in ['\n$', '\n\xa0', '\n', '\xa0']:
            
            n_element = ele_norm.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(n_element)
            
            result = [int(clean_ele) * m for clean_ele in res]

            column_list.extend(result)

    return column_list
    

# Throw column list-like object into cleaning function. 
def cleaning_column(column, m):
     
    column_list = []

    for element_post, element in enumerate(column):
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:
            
            element = unicodedata.normalize('NFKD', element)
            
            element = element.replace( '\n', '')
            
            clean_ele = clean_col_name( element )
            
            pattern = re.compile(r'.+?(?=\()')
            
            if pattern.match(clean_ele):
                
                cleaner_ele = pattern.findall(clean_ele)
                cleaner_ele = cleaner_ele[0]
                column_list.append(cleaner_ele)
                
            else:
                #print(cleaner_ele)
                column_list.append(clean_ele)
            
        # Use a dictionary to convert the Months into numbers.
        #elif element_post == :
            #month = re.sub('({})'.format('|'.join(map(re.escape, month_dict.keys()))), lambda m: month_dict[m.group()], dat)
            #column_list.extend(month)
        #elif element == '(' or ')': 
            #pint(element)
            #continue
            
        else:
            
            element = element.replace( '\n$' and '\n\xa0' and '\n', '0')
            
            element = element.replace( ',', '' )
        
            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')
            
            res = pattern.findall(element)

            res = [int(ele) * m for ele in res]
            
            column_list.extend(res)

    return column_list
    

## Cleaning Tables Function

In [18]:
# New and imporved.
def Quarter( column , k):
    
#     print('column used for quarter', column)
    
    multi_list = [element.split() for element in column if element.split() != []]
#     print('multi_list', multi_list)
    new_column = [item for sublist in multi_list for item in sublist]
#     print('new_column used for year', new_column)
    
    Q = []
    Y = []
    
    for letter in column:
#         print('letter', letter)
        try:
            month = re.findall(r'\w{}+'.format(list(month_dict.keys())), letter.replace(' ',  '') )[0]
            
            Q_num = month_dict[month] 
    #         print('Q_num' , Q_num)
    
            if Q_num != []:
                Q.append(Q_num)
                
        except:
            continue
    #         print('~'*20)
        
#     print('Q' , Q) 
#     Q_final = Q[0] 
#     print('Q_final' , Q_final)
    
  
    for letter in new_column:   
                
        Year = [re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) for p_s_ele_norm in letter.split(',') if re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) != []]
        Y.append(Year)
    print(f'Y - {Y}')   # Y - [[], [], [['2015']], [['2016']]]

    
    Y = [item for sublist in Y for item in sublist ][1]
#     print('this is the interger version of Y - ', int(Y[0]))
    
    if (k == '10-Q' and int(Y[0]) < 2017) or (k=='10-K' and int(Y[0]) < 2016):    
        Y = [str(int(Y[0]) + 1)]
#         print(f'Changed Y into {Y}')
        
    if int(Y[0]) > 2016 or (k=='10-K' and int(Y[0])< 2017):
        Y.insert(0, 0)
       
    #print(f'Y - {Y}')
    
    if int(Y[0]) < 2017:
        Q_final =['Quarter'] + Q
#         Q_final.insert(1, 0)
        
    else:
        Q_final =['Quarter'] + Q
        
    Y.insert(0, 'Year')
    
    
#     print( '\n', 'Q_final', Q_final, 'Y', Y)
    return Q_final, Y

In [19]:
def table_extractor( page, multiplier, k ):
    
    # Convert string to BS object
    soup = BeautifulSoup(page, 'html5')

    
    # Get only the first table in the page. 
    for table_num, table in enumerate(soup.find_all('tbody')):
        if table_num == 0: 
            
            # then get all the rows in the table.
            table_rows = soup.find_all('tr')

            if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
                del table_rows[0]

            single_table = []
            Date_column = []

            # Loop through each column, adding in three structures to single_table.
            for tr_post, tr in enumerate(table_rows):

                td = tr.find_all('td')

                column = [tr.text for tr in td]
                #print( 'tr_post', tr_post, len(column), column)
                # 10-Q 
                if k == '10-Q':

                    if tr_post == 1:

                        # Quarter Column
                        col_Q, col_year = Quarter( column, k )
                        single_table.append( col_Q )
                        # Year Column
                        single_table.append( col_year )

                    elif tr_post > 2: 

                        # Features Columns
                        cleaned_data = cleaning_column( column , multiplier )
                        #print(cleaned_data)
                        if len(cleaned_data) > 10:
                            del cleaned_data[3]
                        if cleaned_data == ['']:
                            break 

                        single_table.append( cleaned_data )

                # 10-K
                elif k == '10-K':

                    if tr_post == 1:

                        # Quarter Column
                        col_Q, col_year = Quarter( column,k )
                        single_table.append( col_Q )
                        # Year Column
                        single_table.append( col_year )

                    elif tr_post > 2:              

                        cleaned_data = cleaning_column( column , 10**6 )
                        if len(cleaned_data) > 10:
                            del cleaned_data[3]
                        if cleaned_data == ['']:
                            break
                            
                        single_table.append( cleaned_data )

            return single_table

In [20]:
def df_clean( single_table ):
    #print(single_table)
    table_df = pd.DataFrame(single_table)
    #print(table_df)
    table_df = table_df.transpose()  
    #print(table_df)
    #table_df.drop(table_df.index[1:3], 0, inplace=True)

    #table_df.drop(table_df.index[2:], 0, inplace=True)

    table_df.columns = table_df.iloc[0]

    table_df = table_df.drop(table_df.index[0])
    
    return table_df

In [21]:
def table(page_code, k):
    
    doc_tables=[]

    for page_post, page in enumerate(page_code):
        
        if anchor_check(page) and ('in millions' in page):
            
            single_table = table_extractor( page, 10**6, k )
            #print(single_table)
            table_df = df_clean(single_table)
                        
            doc_tables.append(table_df)
        
        elif anchor_check(page) and ('in thousands' in page):
            
            #print('page_post', page_post)
            
            single_table = table_extractor( page, 10**3, k )

            table_df = df_clean(single_table)

            doc_tables.append(table_df)
            
#         else:
#             print(page_post+1, 'anchor_check / in thousands/million problem')

                
    return doc_tables
    

# Exploring the Data

In [25]:
# Made for debugging purposes. 
CBS_A = 'CONSOLIDATED BALANCE SHEET'
# CBS_B = 'name="BALANCE_SHEETS"'
# CBS_D = 'BALANCE SHEETS '
CBS_C = 'Cash and cash equivalents'

string_test = master_filings_dict['0000037996-20-000041']['filing_documents']['10-Q']['pages_code']
k = '10-Q'
for page in string_test:
    
    # if the page has one of these anchors
    if (CBS_A in page) and (CBS_C in page):
        
        # Convert string to BS object
        soup = BeautifulSoup(page, 'html5')
        
        # Get only the first table in the page. 
        for table_num, table in enumerate(soup.find_all('tbody')):
            if table_num == 0: 
            
                # then get all the rows in the table.
                table_rows = soup.find_all('tr')

                if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
                    del table_rows[0]

                single_table = []
                Date_column = []

                # Loop through each column, adding in three structures to single_table.
                for tr_post, tr in enumerate(table_rows):

                    td = tr.find_all('td')

                    column = [tr.text for tr in td]
                    #print( 'tr_post', tr_post, len(column), column)
                    # 10-Q 

                    if tr_post == 1:

                            # Quarter Column
                            col_Q, col_year = Quarter( column, k )
                            single_table.append( col_Q )
                            # Year Column
                            single_table.append( col_year )

                    elif tr_post > 2: 
                        print(column)
                        # Features Columns
                        cleaned_data = cleaning_column( column , 10**6 )
                        #print(cleaned_data)
                        if len(cleaned_data) > 10:
                            del cleaned_data[3]
                        
                        if cleaned_data == ['']:
                            break
                        single_table.append( cleaned_data )

                table_df = pd.DataFrame(single_table)
                #print(table_df)
                table_df = table_df.transpose()
                #print(table_df)
            #table_df.drop(table_df.index[1:3], 0, inplace=True)

            #table_df.drop(table_df.index[2:], 0, inplace=True)

            #     table_df.columns = table_df.iloc[0]

            #     table_df = table_df.drop(table_df.index[0])
            else:
                print( '\n', '~'*40, '\n')

Y - [[], [], [['2019']], [], [], [['2020']]]
['ASSETS', '\xa0', '\xa0', '\xa0']
['Cash and cash equivalents (Note 7)', '$', '17,504', '', '\xa0', '$', '25,971', '']
['Marketable securities (Note 7)', '17,147', '', '\xa0', '20,399', '']
['Ford Credit finance receivables, net of allowance for credit losses of $162 and $389 (Note 8)', '53,651', '', '\xa0', '54,889', '']
['Trade and other receivables, less allowances of $63 and $72', '9,237', '', '\xa0', '6,625', '']
['Inventories (Note 9)', '10,786', '', '\xa0', '11,312', '']
['Assets held for sale (Note 17)', '2,383', '', '\xa0', '700', '']
['Other assets', '3,339', '', '\xa0', '4,380', '']
['Total current assets', '114,047', '', '\xa0', '124,276', '']
['\xa0', '\xa0', '\xa0', '\xa0']
['Ford Credit finance receivables, net of allowance for credit losses of $351 and $842 (Note 8)', '53,703', '', '\xa0', '51,141', '']
['Net investment in operating leases', '29,230', '', '\xa0', '28,514', '']
['Net property', '36,469', '', '\xa0', '35,294',

In [22]:
explore_dict = {}
for QK, AccNum_list in QorK.items():
    
    #for an_postition, acc_num in enumerate(AccNum_list): 
    if QK== '10-Q':
        Q_new = AccNum_list[0]
        Q_old = AccNum_list[-1]
        explore_dict[QK] = [Q_new, Q_old]

    if QK == '10-K':
        K_new = AccNum_list[0]
        K_old = AccNum_list[-1]
        explore_dict[QK] = [K_new, K_old]

In [23]:
explore_dict

{'10-Q': ['0000037996-20-000041', '0000037996-13-000027'],
 '10-K': ['0000037996-20-000010', '0000037996-13-000014']}

# Save Files Locally. 

In [115]:
# Checklist of erroneous filings.  
for a, b in Checklist.items():
    if b != []:
        print('These have a problem.', '\n', a, b)

    else: 
        print(a, 'has no problem!! :) ' , '\n')
    
print(Checklist)

{}


In [138]:
# Bring QorK over to Part 2. 
QorK ={'10-Q': ['0000037996-20-000041',
  '0000037996-19-000078',
  '0000037996-19-000067',
  '0000037996-19-000026',
  '0000037996-18-000084',
  '0000037996-18-000063',
  '0000037996-18-000036',
  '0000037996-17-000092',
  '0000037996-17-000068',
  '0000037996-17-000036',
  '0000037996-16-000158',
  '0000037996-16-000133',
  '0000037996-16-000107',
  '0000037996-15-000064',
  '0000037996-15-000046',
  '0000037996-15-000029',
  '0000037996-14-000057',
  '0000037996-14-000043',
  '0000037996-14-000023',
  '0000037996-13-000055',
  '0000037996-13-000042',
  '0000037996-13-000027'],
 '10-K': ['0000037996-20-000010',
  '0000037996-19-000012',
  '0000037996-18-000015',
  '0000037996-17-000013',
  '0000037996-16-000092',
  '0000037996-15-000013',
  '0000037996-14-000010',
  '0000037996-13-000014']}

In [140]:
QorK = {'10-Q': ['0000037996-20-000041',
  '0000037996-19-000078',
  '0000037996-19-000067',
  '0000037996-19-000026',
  '0000037996-18-000084',
  '0000037996-18-000063',
  '0000037996-18-000036',
  '0000037996-17-000092',
  '0000037996-17-000068',
  '0000037996-17-000036'],
 '10-K': ['0000037996-20-000010',
  '0000037996-19-000012',
  '0000037996-18-000015',
  '0000037996-17-000013',
  '0000037996-16-000092',
  '0000037996-15-000013',
  '0000037996-14-000010',
  '0000037996-13-000014']}

In [110]:
QorK = { '10-Q': [ '0000037996-16-000158',
  '0000037996-16-000133',
  '0000037996-16-000107',
  '0000037996-15-000064',
  '0000037996-15-000046',
  '0000037996-15-000029',
  '0000037996-14-000057',
  '0000037996-14-000043',
  '0000037996-14-000023',
  '0000037996-13-000055',
  '0000037996-13-000042',
  '0000037996-13-000027']
}